In [1]:
import torch
import torchaudio
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import glob
import librosa
import librosa.display
from sklearn.linear_model import LogisticRegression
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
import glob
import soundfile as sf


c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\torchaudio\backend\utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 to '


In [2]:
tqdm.pandas(desc="!")


c:\users\bhavi\appdata\local\programs\python\python37\lib\site-packages\tqdm\std.py:699: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
train_classified = pd.read_pickle("train_classified")
train_classified["length"] = 0

In [4]:
train_classified

,sample_rate,species_id,waveform,length
0,48000.0,14.0,"[0.001678466796875, 0.0067138671875, -0.002929...",0
1,48000.0,23.0,"[0.021209716796875, 0.018310546875, 0.01879882...",0
2,48000.0,12.0,"[-0.011444091796875, -0.000396728515625, 0.015...",0
3,48000.0,17.0,"[0.0008544921875, 0.001953125, 0.0043334960937...",0
4,48000.0,10.0,"[0.0009765625, 0.002899169921875, 0.0007934570...",0
...,...,...,...,...
1211,48000.0,13.0,"[0.015350341796875, 0.032379150390625, 0.02734...",0
1212,48000.0,4.0,"[-0.03717041015625, -0.024932861328125, -0.008...",0
1213,48000.0,0.0,"[0.019012451171875, 0.015106201171875, 0.00924...",0
1214,48000.0,5.0,"[-0.014678955078125, -0.009765625, -0.01284790...",0


In [5]:

for i in range(len(train_classified)):
    train_classified.iloc[i, train_classified.columns.get_loc("length")] = len(train_classified.iloc[i]["waveform"])

In [6]:
train_classified.species_id.unique().astype(int).astype(str)

array(['14', '23', '12', '17', '10', '8', '0', '18', '15', '1', '6', '7',
       '11', '13', '16', '19', '3', '2', '4', '22', '9', '21', '20', '5'],
      dtype='<U11')

In [7]:

len(train_classified.iloc[2]['waveform'])

150529

In [8]:
# cut off at 44.1k sample rate /. 2. 
# mapping from time domain into frequency domain
# https://www.nti-audio.com/en/support/know-how/fast-fourier-transform-fft

# y = f(x)

def extract_features( row ):
    samplerate = 48000
    data = row['waveform']
    varfft = np.abs( np.fft.fft(a=data)[:(len(data)//2)] )
    x = np.linspace(0, len(varfft), num=len(varfft), endpoint=True)
    f1 = interp1d(x, varfft, kind='cubic')
    x = np.linspace(0, len(varfft), num=1000, endpoint=True)
    varfft = f1(x)
    
    return varfft

In [9]:
train_classified['fft_1d'] = train_classified.progress_apply(extract_features, axis=1)

In [10]:
TRAIN = []
TARGET = []
for i in range(len(train_classified)):
    entry = train_classified["fft_1d"].iloc[i]
    t = train_classified["species_id"].iloc[i]
    TRAIN.append(entry)
    TARGET.append(t)

In [11]:
# Basic Log Reg
X = train_classified.loc[:, train_classified.columns == "fft_1d"]
y = train_classified.loc[:, train_classified.columns == "species_id"]

In [12]:

X_train, X_test, y_train, y_test = train_test_split(TRAIN, TARGET)


In [13]:
logreg = LogisticRegression( max_iter=5000 )
logreg.fit(TRAIN, TARGET)

LogisticRegression(max_iter=5000)

In [14]:
logreg.score(X_test, y_test)

1.0

In [15]:
testfiles = glob.glob( 'data/test/*.flac' )

In [16]:
testfiles

['data/test\\000316da7.flac',
 'data/test\\003bc2cb2.flac',
 'data/test\\0061c037e.flac',
 'data/test\\010eb14d3.flac',
 'data/test\\011318064.flac',
 'data/test\\01307e693.flac',
 'data/test\\013bd8b85.flac',
 'data/test\\01812f522.flac',
 'data/test\\019db5220.flac',
 'data/test\\01bd97dd6.flac',
 'data/test\\01d36def5.flac',
 'data/test\\01fefebcb.flac',
 'data/test\\0208fec47.flac',
 'data/test\\0232841d4.flac',
 'data/test\\02c30604d.flac',
 'data/test\\02dbcc383.flac',
 'data/test\\0303dee3c.flac',
 'data/test\\034f46152.flac',
 'data/test\\037460d90.flac',
 'data/test\\039395460.flac',
 'data/test\\039892ae3.flac',
 'data/test\\03a06a709.flac',
 'data/test\\03a9a5b2a.flac',
 'data/test\\03d0c7428.flac',
 'data/test\\03f5a4505.flac',
 'data/test\\03f8931a1.flac',
 'data/test\\04053cd3e.flac',
 'data/test\\040d4f0d5.flac',
 'data/test\\042a6c838.flac',
 'data/test\\044173e0d.flac',
 'data/test\\04496d53a.flac',
 'data/test\\0465bce22.flac',
 'data/test\\047a7c4bf.flac',
 'data/tes

In [17]:
testfiles[1][10:19]

'003bc2cb2'

In [18]:
test_df = pd.DataFrame()

In [19]:
for i in tqdm(range(len(testfiles) // 2)):
    waveform, sample_rate = sf.read(testfiles[i])
    test_df=test_df.append({"waveform" : waveform, "recording_id" : testfiles[i][10:19]}, ignore_index=True)
    

In [20]:
test_df

,recording_id,waveform
0,000316da7,"[-0.002532958984375, 0.00115966796875, -0.0045..."
1,003bc2cb2,"[0.062347412109375, -0.02972412109375, 0.00027..."
2,0061c037e,"[-0.005126953125, -0.004608154296875, -0.00811..."
3,010eb14d3,"[-0.02447509765625, 0.011688232421875, 0.03973..."
4,011318064,"[0.00128173828125, -0.002227783203125, 0.00207..."
...,...,...
991,7cfaed5a0,"[-0.013153076171875, -0.0196533203125, -0.0148..."
992,7d064b789,"[0.010284423828125, 0.01513671875, 0.022003173..."
993,7d188d367,"[-0.001068115234375, -0.00244140625, 0.0063476..."
994,7d4c75228,"[0.009063720703125, 0.0244140625, 0.0302429199..."


In [21]:

test_slice_df = pd.DataFrame()

In [22]:
# x + 121781
# x = [0 -> length- 121781]


for i in tqdm(range(len(test_df) // 3)):
    entry = test_df.iloc[i]
    x = 0
    c_length =  len(entry['waveform']) if len(entry['waveform']) < 121781 else 121781
    while x < len(entry['waveform']) - c_length:
        test_slice_df = test_slice_df.append({"waveform": waveform[x : x+c_length], "recording_id" : entry['recording_id']}, ignore_index=True)
        
        x = x+ c_length

In [23]:
test_slice_df

,recording_id,waveform
0,000316da7,"[0.008331298828125, 0.0130615234375, 0.0155334..."
1,000316da7,"[0.024078369140625, 0.01800537109375, 0.002044..."
2,000316da7,"[0.0009765625, -0.0035400390625, -0.0032653808..."
3,000316da7,"[-0.013153076171875, -0.004730224609375, -0.00..."
4,000316da7,"[0.003997802734375, 0.005218505859375, 0.00845..."
...,...,...
7631,2d02cb72a,"[0.00494384765625, 0.0126953125, 0.00628662109..."
7632,2d02cb72a,"[0.003814697265625, 0.000213623046875, 0.00411..."
7633,2d02cb72a,"[0.00921630859375, -0.007537841796875, -0.0095..."
7634,2d02cb72a,"[-0.016815185546875, -0.015289306640625, -0.00..."


In [24]:
test_slice_df['fft_1d'] = test_slice_df.progress_apply(extract_features, axis=1)

In [25]:
test_slice_df

,recording_id,waveform,fft_1d
0,000316da7,"[0.008331298828125, 0.0130615234375, 0.0155334...","[8.457885742187496, 5.571576067748188, 21.8049..."
1,000316da7,"[0.024078369140625, 0.01800537109375, 0.002044...","[4.29119873046875, 2.318464001644091, 7.694154..."
2,000316da7,"[0.0009765625, -0.0035400390625, -0.0032653808...","[4.696807861328127, 1.3955767563103598, 8.5942..."
3,000316da7,"[-0.013153076171875, -0.004730224609375, -0.00...","[3.437530517578124, 5.788097434872375, 8.72308..."
4,000316da7,"[0.003997802734375, 0.005218505859375, 0.00845...","[1.8282470703124953, 2.7084323128504777, 5.525..."
...,...,...,...
7631,2d02cb72a,"[0.00494384765625, 0.0126953125, 0.00628662109...","[2.5891418457031268, 2.624587558146056, 12.051..."
7632,2d02cb72a,"[0.003814697265625, 0.000213623046875, 0.00411...","[3.4154052734374996, 3.4470006644419064, 9.903..."
7633,2d02cb72a,"[0.00921630859375, -0.007537841796875, -0.0095...","[2.462371826171869, 2.154273678572248, 7.60016..."
7634,2d02cb72a,"[-0.016815185546875, -0.015289306640625, -0.00...","[2.620208740234373, 3.595164874555891, 3.82042..."


In [26]:
TEST = []
for i in range(len(test_slice_df)):
    entry = test_slice_df["fft_1d"].iloc[i]
    TEST.append(entry)


In [28]:
ids = []
for i in range(len(test_slice_df)):
    entry = test_slice_df["recording_id"].iloc[i]
    ids.append(entry)

In [29]:
len(ids)

7636

In [30]:
Y_pp = pd.DataFrame(logreg.predict_proba(TEST), 
             columns=['s14', 's23', 's12', 's17', 's10', 's8', 's0', 's18', 's15', 's1', 's6', 's7',
       's11', 's13', 's16', 's19', 's3', 's2', 's4', 's22', 's9', 's21', 's20', 's5'])
# Y_pp['recording_id'] = ids
Y_pp.tail()

,s14,s23,s12,s17,s10,s8,s0,s18,s15,s1,...,s16,s19,s3,s2,s4,s22,s9,s21,s20,s5
7631,6.159591e-11,3.720124e-11,4.026498e-10,3.332567e-14,3.586588e-09,9.983836e-01,3.049226e-13,1.186365e-04,3.196112e-09,4.301196e-09,...,0.000003,7.233939e-08,1.948406e-20,1.391249e-15,5.967126e-05,1.611087e-09,1.209721e-04,2.477826e-16,1.018053e-05,3.641627e-11
7632,2.184086e-07,4.121856e-08,5.972119e-08,7.806062e-08,5.128886e-06,9.946841e-01,4.662851e-10,1.775234e-03,1.570007e-05,7.268756e-08,...,0.000708,1.750481e-04,2.035256e-14,7.218818e-09,2.113835e-05,3.211588e-09,7.286287e-04,1.941459e-12,1.723099e-05,3.768295e-08
7633,4.955069e-07,7.701422e-08,1.596654e-05,4.566181e-03,5.878148e-08,7.898996e-01,1.221148e-11,2.719492e-03,5.744234e-06,3.277273e-06,...,0.064071,1.714062e-03,4.831315e-13,5.252197e-10,1.882498e-02,9.708309e-07,1.062891e-01,2.889040e-14,8.758362e-03,4.691329e-07
7634,1.115713e-18,2.209001e-01,4.734750e-17,2.058655e-17,4.149309e-13,2.390268e-10,3.197242e-18,1.608957e-12,6.348570e-05,1.600341e-13,...,0.779036,3.990811e-14,1.409709e-24,2.000556e-16,1.551261e-16,2.288338e-20,2.400231e-08,7.818172e-29,2.356783e-13,1.361870e-14
7635,2.509933e-09,5.640805e-08,1.488536e-06,1.230787e-05,4.207719e-09,9.984540e-01,9.621210e-11,6.915952e-05,1.190678e-06,5.599988e-08,...,0.000307,6.242708e-07,1.852966e-13,8.071792e-07,5.962916e-07,1.693570e-08,8.307187e-04,4.355466e-14,2.140949e-05,2.891967e-07


In [31]:
softmax = Y_pp.idxmax(axis=1)

In [32]:
softmax

0        s8
1        s8
2        s8
3       s16
4        s8
       ... 
7631     s8
7632     s8
7633     s8
7634    s16
7635     s8
Length: 7636, dtype: object